In [ ]:
!pip install supabase

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from supabase import create_client, Client
from datetime import datetime

In [ ]:
import os

SUPABASE_URL = "Enter your Supabase URL"
SUPABASE_KEY = "Enter your Supabase Service key"

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)


In [ ]:
# Fetch data in batches from Supabase
batch_size = 1000
offset = 0
all_data = []

while True:
    response = (
        supabase
        .table("air_quality_data")
        .select("*")
        .range(offset, offset + batch_size - 1)
        .execute()
    )

    batch = response.data
    if not batch:  # No more data
        break

    all_data.extend(batch)
    offset += batch_size

# Convert to DataFrame
df = pd.DataFrame(all_data)

# Convert datetime columns
df['datetime_utc'] = pd.to_datetime(df['datetime_utc'])
df['datetime_ist'] = pd.to_datetime(df['datetime_ist'])

print(f"Loaded {len(df)} rows from Supabase")
df.head()


Loaded 7179 rows from Supabase


,id,city,datetime_utc,datetime_ist,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,pressure_msl,surface_pressure,...,pm10,pm2_5,carbon_monoxide,carbon_dioxide,nitrogen_dioxide,sulphur_dioxide,ozone,uv_index,uv_index_clear_sky,methane
0,1,Delhi,2025-05-09 18:30:00,2025-05-10 00:00:00,27.9,67.0,21.1,32.0,1005.8,980.8,...,230.0,65.4,1039,483,33.5,31.9,72,0.0,0.0,1598
1,2,Delhi,2025-05-09 19:30:00,2025-05-10 01:00:00,27.5,65.0,20.2,30.8,1005.1,980.1,...,230.0,66.4,985,487,33.5,31.4,67,0.0,0.0,1602
2,3,Delhi,2025-05-09 20:30:00,2025-05-10 02:00:00,27.2,69.0,20.9,31.0,1004.8,979.8,...,221.2,66.8,944,491,34.4,31.1,61,0.0,0.0,1606
3,4,Delhi,2025-05-09 21:30:00,2025-05-10 03:00:00,26.8,69.0,20.7,30.6,1004.9,979.9,...,209.5,66.6,930,495,37.7,30.6,50,0.0,0.0,1611
4,5,Delhi,2025-05-09 22:30:00,2025-05-10 04:00:00,26.1,68.0,19.6,29.0,1005.3,980.2,...,206.3,67.5,928,500,41.8,30.4,38,0.0,0.0,1615


In [ ]:
df.columns

Index(['id', 'city', 'datetime_utc', 'datetime_ist', 'temperature_2m',
       'relative_humidity_2m', 'dew_point_2m', 'apparent_temperature',
       'pressure_msl', 'surface_pressure', 'cloudcover', 'windspeed_10m',
       'winddirection_10m', 'pm10', 'pm2_5', 'carbon_monoxide',
       'carbon_dioxide', 'nitrogen_dioxide', 'sulphur_dioxide', 'ozone',
       'uv_index', 'uv_index_clear_sky', 'methane'],
      dtype='object')

In [ ]:
# Example: fill numeric missing values with median
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# If needed: drop rows with essential missing datetime
df = df.dropna(subset=['datetime_utc', 'datetime_ist'])

print("Missing values after handling:")
print(df.isna().sum())


Missing values after handling:
id                      0
city                    0
datetime_utc            0
datetime_ist            0
temperature_2m          0
relative_humidity_2m    0
dew_point_2m            0
apparent_temperature    0
pressure_msl            0
surface_pressure        0
cloudcover              0
windspeed_10m           0
winddirection_10m       0
pm10                    0
pm2_5                   0
carbon_monoxide         0
carbon_dioxide          0
nitrogen_dioxide        0
sulphur_dioxide         0
ozone                   0
uv_index                0
uv_index_clear_sky      0
methane                 0
dtype: int64


In [ ]:
# Function to calculate sub-index
def calc_aqi_subindex(Cp, breakpoints):
    for (Clow, Chigh, Ilow, Ihigh) in breakpoints:
        if Clow <= Cp <= Chigh:
            return ((Ihigh - Ilow) / (Chigh - Clow)) * (Cp - Clow) + Ilow
    return None

# CPCB breakpoints
breakpoints_pm25 = [(0, 30, 0, 50), (31, 60, 51, 100), (61, 90, 101, 200),
                    (91, 120, 201, 300), (121, 250, 301, 400), (251, 500, 401, 500)]
breakpoints_pm10 = [(0, 50, 0, 50), (51, 100, 51, 100), (101, 250, 101, 200),
                    (251, 350, 201, 300), (351, 430, 301, 400), (431, 500, 401, 500)]
breakpoints_no2 = [(0, 40, 0, 50), (41, 80, 51, 100), (81, 180, 101, 200),
                   (181, 280, 201, 300), (281, 400, 301, 400), (401, 500, 401, 500)]
breakpoints_so2 = [(0, 40, 0, 50), (41, 80, 51, 100), (81, 380, 101, 200),
                   (381, 800, 201, 300), (801, 1600, 301, 400), (1601, 5000, 401, 500)]
breakpoints_o3  = [(0, 50, 0, 50), (51, 100, 51, 100), (101, 168, 101, 200),
                   (169, 208, 201, 300), (209, 748, 301, 400), (749, 1000, 401, 500)]

# Calculate sub-indices
df['aqi_pm25'] = df['pm2_5'].apply(lambda x: calc_aqi_subindex(x, breakpoints_pm25))
df['aqi_pm10'] = df['pm10'].apply(lambda x: calc_aqi_subindex(x, breakpoints_pm10))
df['aqi_no2']  = df['nitrogen_dioxide'].apply(lambda x: calc_aqi_subindex(x, breakpoints_no2))
df['aqi_so2']  = df['sulphur_dioxide'].apply(lambda x: calc_aqi_subindex(x, breakpoints_so2))
df['aqi_o3']   = df['ozone'].apply(lambda x: calc_aqi_subindex(x, breakpoints_o3))

# Final AQI
df['AQI'] = df[['aqi_pm25', 'aqi_pm10', 'aqi_no2', 'aqi_so2', 'aqi_o3']].max(axis=1)

# Dominant pollutant
def find_dominant(row):
    sub_indices = {
        'PM2.5': row['aqi_pm25'],
        'PM10': row['aqi_pm10'],
        'NO2': row['aqi_no2'],
        'SO2': row['aqi_so2'],
        'O3': row['aqi_o3']
    }
    return max(sub_indices, key=sub_indices.get)

df['dominant_pollutant'] = df.apply(find_dominant, axis=1)

# AQI Category
def aqi_category(aqi):
    if aqi <= 50: return "Good"
    elif aqi <= 100: return "Satisfactory"
    elif aqi <= 200: return "Moderate"
    elif aqi <= 300: return "Poor"
    elif aqi <= 400: return "Very Poor"
    else: return "Severe"

df['AQI_Category'] = df['AQI'].apply(aqi_category)

df[['datetime_ist', 'AQI', 'AQI_Category', 'dominant_pollutant']].head()


,datetime_ist,AQI,AQI_Category,dominant_pollutant
0,2025-05-10 00:00:00,186.711409,Moderate,PM10
1,2025-05-10 01:00:00,186.711409,Moderate,PM10
2,2025-05-10 02:00:00,180.864430,Moderate,PM10
3,2025-05-10 03:00:00,173.090604,Moderate,PM10
4,2025-05-10 04:00:00,170.964430,Moderate,PM10


In [ ]:
df.head()

,id,city,datetime_utc,datetime_ist,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,pressure_msl,surface_pressure,...,uv_index_clear_sky,methane,aqi_pm25,aqi_pm10,aqi_no2,aqi_so2,aqi_o3,AQI,dominant_pollutant,AQI_Category
0,1,Delhi,2025-05-09 18:30:00,2025-05-10 00:00:00,27.9,67.0,21.1,32.0,1005.8,980.8,...,0.0,1598,116.020690,186.711409,41.875000,39.875,72.0,186.711409,PM10,Moderate
1,2,Delhi,2025-05-09 19:30:00,2025-05-10 01:00:00,27.5,65.0,20.2,30.8,1005.1,980.1,...,0.0,1602,119.434483,186.711409,41.875000,39.250,67.0,186.711409,PM10,Moderate
2,3,Delhi,2025-05-09 20:30:00,2025-05-10 02:00:00,27.2,69.0,20.9,31.0,1004.8,979.8,...,0.0,1606,120.800000,180.864430,43.000000,38.875,61.0,180.864430,PM10,Moderate
3,4,Delhi,2025-05-09 21:30:00,2025-05-10 03:00:00,26.8,69.0,20.7,30.6,1004.9,979.9,...,0.0,1611,120.117241,173.090604,47.125000,38.250,50.0,173.090604,PM10,Moderate
4,5,Delhi,2025-05-09 22:30:00,2025-05-10 04:00:00,26.1,68.0,19.6,29.0,1005.3,980.2,...,0.0,1615,123.189655,170.964430,52.005128,38.000,38.0,170.964430,PM10,Moderate


In [ ]:
# === New Feature Engineering ===
def create_features(df, lags=24):
    df = df.copy()

    # Lag Features
    for lag in range(1, lags+1):
        df[f"lag_{lag}"] = df["AQI"].shift(lag)

    # Rolling Statistics
    df["rolling_mean_6h"] = df["AQI"].rolling(window=6).mean()
    df["rolling_mean_12h"] = df["AQI"].rolling(window=12).mean()
    df["rolling_mean_24h"] = df["AQI"].rolling(window=24).mean()

    df["rolling_std_6h"] = df["AQI"].rolling(window=6).std()
    df["rolling_std_12h"] = df["AQI"].rolling(window=12).std()
    df["rolling_std_24h"] = df["AQI"].rolling(window=24).std()

    # Differences
    df["aqi_diff_1h"] = df["AQI"].diff(1)
    df["aqi_diff_6h"] = df["AQI"].diff(6)

    # Time Features
    df["hour"] = df["datetime_utc"].dt.hour
    df["dayofweek"] = df["datetime_utc"].dt.dayofweek
    df["month"] = df["datetime_utc"].dt.month

    return df.dropna()

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import pickle
import os

# === Feature Engineering ===
def create_features(df, lags=24):
    df = df.copy()

    # Lag Features
    for lag in range(1, lags+1):
        df[f"lag_{lag}"] = df["AQI"].shift(lag)

    # Rolling Statistics
    df["rolling_mean_6h"] = df["AQI"].rolling(window=6).mean()
    df["rolling_mean_12h"] = df["AQI"].rolling(window=12).mean()
    df["rolling_mean_24h"] = df["AQI"].rolling(window=24).mean()

    df["rolling_std_6h"] = df["AQI"].rolling(window=6).std()
    df["rolling_std_12h"] = df["AQI"].rolling(window=12).std()
    df["rolling_std_24h"] = df["AQI"].rolling(window=24).std()

    # Differences
    df["aqi_diff_1h"] = df["AQI"].diff(1)
    df["aqi_diff_6h"] = df["AQI"].diff(6)

    # Time Features
    df["hour"] = df["datetime_utc"].dt.hour
    df["dayofweek"] = df["datetime_utc"].dt.dayofweek
    df["month"] = df["datetime_utc"].dt.month

    return df.dropna()

# === Training Function with Hyperparameter Search ===
def train_city_model(city_df, city_name, save_dir="models", horizons=[1, 2, 3]):
    os.makedirs(save_dir, exist_ok=True)
    results = []

    city_df = create_features(city_df, lags=24)
    X = city_df.drop(columns=["datetime_utc", "AQI"])
    y = city_df["AQI"]

    split_idx = int(len(X) * 0.8)

    for h in horizons:
        y_shifted = y.shift(-h).dropna()
        X_shifted = X.iloc[:len(y_shifted)]

        X_train, X_test = X_shifted.iloc[:split_idx], X_shifted.iloc[split_idx:]
        y_train, y_test = y_shifted.iloc[:split_idx], y_shifted.iloc[split_idx:]

        train_pool = Pool(X_train, y_train)
        test_pool = Pool(X_test, y_test)

        model = CatBoostRegressor(random_state=42, verbose=0)

        # Small random search space (fast)
        param_grid = {
            "depth": [4, 6, 8],
            "learning_rate": [0.05, 0.1, 0.2],
            "iterations": [200, 400],
            "l2_leaf_reg": [3, 5],
            "subsample": [0.8, 1.0]
        }

        search = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_grid,
            n_iter=10,  # only 10 random combos
            cv=3,
            scoring="neg_root_mean_squared_error",
            verbose=0,
            n_jobs=-1,
            random_state=42
        )

        search.fit(X_train, y_train)

        best_model = search.best_estimator_
        y_pred = best_model.predict(X_test)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))

        # Print clean output
        print(f"🏙️ {city_name} Horizon {h}")
        print(f"   Best Params: {search.best_params_}")
        print(f"   MAE: {mae:.2f}, RMSE: {rmse:.2f}\n")

        # Save model
        model_path = os.path.join(save_dir, f"{city_name.lower()}_h{h}_catboost.pkl")
        with open(model_path, "wb") as f:
            pickle.dump(best_model, f)

        results.append({
            "City": city_name,
            "Horizon": h,
            "Best_Params": search.best_params_,
            "MAE": mae,
            "RMSE": rmse,
            "Model_Path": model_path
        })

    return pd.DataFrame(results)

# === Example Run ===
cities = ["Delhi", "Mumbai", "Hyderabad"]
all_results = pd.DataFrame()
for city in cities:
  city_df = df[df["city"] == city][["datetime_utc", "AQI"]].dropna()
  city_results = train_city_model(city_df, city, save_dir="models")
  all_results = pd.concat([all_results, city_results], ignore_index=True)

print(all_results)


🏙️ Delhi Horizon 1
   Best Params: {'subsample': 0.8, 'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 400, 'depth': 4}
   MAE: 31.83, RMSE: 48.33

🏙️ Delhi Horizon 2
   Best Params: {'subsample': 0.8, 'learning_rate': 0.05, 'l2_leaf_reg': 3, 'iterations': 200, 'depth': 4}
   MAE: 48.79, RMSE: 66.14

🏙️ Delhi Horizon 3
   Best Params: {'subsample': 0.8, 'learning_rate': 0.05, 'l2_leaf_reg': 3, 'iterations': 200, 'depth': 4}
   MAE: 60.03, RMSE: 78.07

🏙️ Mumbai Horizon 1
   Best Params: {'subsample': 0.8, 'learning_rate': 0.2, 'l2_leaf_reg': 5, 'iterations': 200, 'depth': 4}
   MAE: 3.78, RMSE: 5.73

🏙️ Mumbai Horizon 2
   Best Params: {'subsample': 0.8, 'learning_rate': 0.2, 'l2_leaf_reg': 5, 'iterations': 200, 'depth': 4}
   MAE: 6.12, RMSE: 9.00

🏙️ Mumbai Horizon 3
   Best Params: {'subsample': 0.8, 'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 400, 'depth': 4}
   MAE: 8.03, RMSE: 11.87

🏙️ Hyderabad Horizon 1
   Best Params: {'subsample': 0.8, 'learning_rate': 0.2, 